In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Wednesday, February 26, 2025
Last Modified: Wednesday, August 13, 2025
Program Purpose: To create the plot which shows the standard deviation in IAV contribution across the models.
"""

In [ ]:
#Importing the appropriate libraries
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import netCDF4 as nc
import glob
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#Making font times
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block is concerned with the calculation of the IAV values for the models, to later be investigated for the standard deviation in IAV

# Area file for adjustments
area_file = np.loadtxt('halfdeg_grid_area.dat')

# Base K&G Calculations.
from collections import OrderedDict
all_datasets_IAV = OrderedDict()

# Load and process all of the files.
pattern = "fco2*"
files = glob.glob(pattern)
name_remove = ["fco2_", "_Dec2022-ext3_1970-2021_yearlymean_XYT.nc"]

for file in files:
    # Getting a name for the files
    file_name = file
    for remove in name_remove:
        file_name = file_name.replace(remove, "")
    file_name = file_name.replace("-", "_").lower()

    # Loading the file and grabbing all of the terrestrial fluxes.
    dataset = nc.Dataset(file, 'r')
    terrestrial_flux = dataset.variables['Terrestrial_flux'][:].astype(np.float64) * 8760  # Convert units to KgC/m2/year from KgC/m2/hr
    terrestrial_flux *= area_file  # Apply the grid cell area adjustment so that fluxes are based on the grid cell size. 10^7 to 10^9
    terrestrial_flux *= 1e-12 #This is the correct value.
    terrestrial_flux = np.flip(terrestrial_flux, axis=1) # Flipping the terrestrial flux data so that it is correctly oriented.
    dataset.close()

    # Removing the wrongly high values from the Classic dataset
    if file_name == "classic_s3":
        threshold = 1.86e+34 # Very high value close to fill value but not quite there.
        terrestrial_flux[terrestrial_flux > threshold] = 0 # Replacing with zero.

    # Removing the water values from the yibs that are not blocked out
    if file_name == "yibs_s3":
        data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
        data_modis_3d= data_modis[np.newaxis, :, :] # Making the data 3d
        data_modis_52= np.repeat(data_modis_3d, 52, axis=0)
        mask = data_modis_52 != 0
        terrestrial_flux = np.ma.masked_array(terrestrial_flux, mask=~mask)

    # Finding the average flux of each grid cell over the 52 years.
    overall_mean_flux_by_grid = terrestrial_flux.mean(axis=0)

    # Finding the flux anomaly for each cell for every year.
    flux_anomaly_yearly_by_grid = terrestrial_flux - overall_mean_flux_by_grid

    # Calculating the yearly global flux anomay by summing all of the individual flux anomalies for each grid cell for that year.
    yearly_global_flux_anomaly = list()
    for i in range(52):
        yearly_global_flux_anomaly.append(np.sum(flux_anomaly_yearly_by_grid[i,:,:]))

    # Finding the bottom sum for the equation (absolute value of global flux anomaly over the years)
    bottom_sum = np.sum(np.abs(yearly_global_flux_anomaly))

    # New array to append top values of the equation to.
    placeholder_array_top_values = np.empty_like(terrestrial_flux)

    for i in range(52):
        # Avoiding division by zero
        if yearly_global_flux_anomaly[i] != 0:
            placeholder_array_top_values[i] = flux_anomaly_yearly_by_grid[i] * (np.abs(yearly_global_flux_anomaly[i]) / yearly_global_flux_anomaly[i])
        else:
            placeholder_array_top_values[i] = 0

    summed_top_values = np.sum(placeholder_array_top_values, axis=0)  # Summing the values for the top of the array
    placeholder_array = summed_top_values / bottom_sum  # Dividing the top and bottom of the equation
    placeholder_array *= 100 ##Multiplying by 100 to make %

    # Adding the placeholder array to the list of IAV maps.
    all_datasets_IAV[file_name] = placeholder_array


In [ ]:
#This block is concerend with the calculation of the standard deviation and the subsequent creation of the map of the standard deviation

#Getting the standard deviation
stackedArray = np.stack(list(all_datasets_IAV.values()), axis=0)
print(np.shape(stackedArray)) #Verifying the shape

#Finding the standard deviation
sd_array = np.std(stackedArray, axis=0)
print(np.shape(sd_array)) #Verifying the shape 

#Masking out the coastal areas to be aligned with the MODIS data
modis_data = np.load("MODIS_Map.npy")
sd_array[modis_data == 0] = np.nan #Where there is water setting the SD in flux to zero


In [ ]:
#This block handles the mapping of the standard deviation on the globe

#Plotting the figure
fig, ax = plt.subplots(figsize=(40, 20), subplot_kw={'projection': ccrs.PlateCarree()})
im = ax.imshow(sd_array, cmap='Reds', origin='upper', extent=[-180, 180, -90, 90])
ax.set_extent([-180, 180, -60, 90], crs=ccrs.PlateCarree()) #Setting the extent of the map such that antarctica is excluded

#Adding the borders and coast
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
gl.left_labels = False
gl.top_labels = False
gl.ylabel_style = {'size': 45}
gl.xlabel_style = {'size': 45}

# Adding the colorbar to the left
cb = plt.colorbar(im, ax=ax, orientation='vertical', pad=0.02)

#Setting the params of the colorbar to make it look nice
cb.ax.ticklabel_format(style='sci', scilimits=(0,0), axis='y')
cb.ax.set_position([ax.get_position().x0 - 0.02, ax.get_position().y0, 0.02, ax.get_position().height]) #Seting the alignment in line with the map
cb.ax.yaxis.offsetText.set_fontsize(60)  
cb.set_label('Standard Deviation of IAV Contribution', fontsize=50)
cb.ax.yaxis.set_ticks_position('left')
cb.ax.yaxis.set_label_position('left')
cb.ax.tick_params(labelsize=56, width=6)

#Sbow plot
plt.show()
